# 공용 함수
- 수정하지 않는 것을 추천드립니다

In [1]:
from jy import get_all
import pandas as pd
from neuralprophet import NeuralProphet

In [2]:
# 상대경로 지정 없이 원본 csv 임포트
main_df = get_all()
main_df_columns = main_df.columns
date_df = main_df.loc[:,"date"]

In [3]:
# 각 지역구와 관련된 변수들
# common은 지역 구분 없는 전체 변수
gw_vars = main_df.loc[:,main_df_columns[2:9]]
ddm_vars = main_df.loc[:,main_df_columns[9:16]]
sd_vars = main_df.loc[:,main_df_columns[16:23]]
jl_vars = main_df.loc[:,main_df_columns[23:30]]
common_vars = main_df.loc[:,main_df_columns[30:46]]

In [4]:
# 각 지역구의 따릉이 수요량 
gw_demand=main_df.loc[:,main_df_columns[46]]
ddm_demand=main_df.loc[:,main_df_columns[47]]
sd_demand=main_df.loc[:,main_df_columns[48]]
jl_demand=main_df.loc[:,main_df_columns[49]]

In [5]:
# 날짜와 수요량 컬럼을 원하는 대로 정리해주는 함수
def create_y (df_demand, date_name, y_name):
    result = pd.concat([date_df, df_demand], axis=1)
    result = result.rename(columns={"date":date_name, result.columns[1]:y_name})
    return result

In [6]:
# 변수들을 일부 샘플링한 dataframe의 리스트를 반환하는 함수
## amount는 (수요량 이외에) 추가로 샘플링할 변수의 개수 (0, 1, 2 가능. 0은 수요량만 반환)
## df_y는 날짜와 수요량이 포함된 데이터프레임. create_y 함수로 정제한 값을 넣으면 됩니다
## df_list는 추가 변수로 사용할 모든 데이터프레임들의 리스트. ~~~_vars 중 원하는 것을 리스트로 넣으면 됩니다
    ## 예시) 광진구 + 공용 변수의 경우: df_list=[gw_vars, comon_vars]
## 반환하는 값은 튜플 {"data": 데이터프레임, "name":포함된 컬럼명의 리스트} 로 이루어진 리스트입니다
    
def sample_multivariate(amount, df_y, df_list):
    samples = []
    # 데이터프레임을 하나로 통합
    df = pd.concat(df_list, axis=1)
    if amount == 0:
        # y값만 리턴
        samples.append({"data":pd.concat([df_y], axis=1),"name":[]})
    elif amount == 1:
        # sample을 한 column씩 추출
        for index in range(len(df.columns)):
            sample = pd.concat([df_y,df.loc[:,df.columns[index]]],axis=1)
            samples.append({"data":sample, "name":[df.columns[index]]})
    elif amount == 2:
        # sample을 두 column씩 추출
        for first_index in range(len(df.columns)):
            remaining_columns = df.columns[first_index+1:]
            for second_index in range(len(remaining_columns)):
                targets = [df.columns[first_index],remaining_columns[second_index]]
                sample = pd.concat([df_y,df.loc[:,targets]],axis=1)
                samples.append({"data":sample, "name":targets})
    return samples
    

In [7]:
# 데이터를 자르는 함수
# 일단은 2021년 1월 1일 기준으로 자르게 일반화 해놓았습니다

def split_sample(df):
    train = df.loc[:1095]
    test = df.loc[1096:]
    return {"train":train, "test":test}


In [8]:
def create_result(indexes):
    return pd.DataFrame(index=["Column"] + indexes)

공용 함수 End

In [11]:
sample = create_y(gw_demand,"ds","y")

In [12]:
split = split_sample(sample)
train = split["train"]
test = split["test"]

In [21]:
m = NeuralProphet( growth='linear', # 추세 유형 설정(linear, discontinuous, off 중 선택 가능)
        yearly_seasonality=True, #년간 계절성 설정
        weekly_seasonality=True, #주간 계절성 설정
        daily_seasonality=True, #일간 계절성 설정
        batch_size=32,#배치 사이즈 설정
        epochs=30,#학습 횟수 설정
        learning_rate=0.1, # 학습률 설정
)

m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
metrics = m.fit(train, freq='d',validation_df=test)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.909% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - d
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [31]:
future = m.make_future_dataframe(sample, periods=365)
future

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.932% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - d
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


,ds,y
0,2022-01-01,None
1,2022-01-02,None
2,2022-01-03,None
3,2022-01-04,None
4,2022-01-05,None
...,...,...
360,2022-12-27,None
361,2022-12-28,None
362,2022-12-29,None
363,2022-12-30,None


In [23]:
forecast = m.predict(df=future)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.726% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - d
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.726% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - d


Predicting: 35it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [25]:
forecast

,ds,y,yhat1,trend,season_yearly,season_weekly,season_daily,season_monthly
0,2021-01-01,None,2.891697,-2.754649,-3.603158,0.156855,9.082088,0.010560
1,2021-01-02,None,2.906502,-2.756171,-3.612601,0.100525,9.082088,0.092662
2,2021-01-03,None,2.607873,-2.757692,-3.620431,-0.279687,9.082088,0.183596
3,2021-01-04,None,2.837090,-2.759214,-3.626808,-0.016915,9.082088,0.157939
4,2021-01-05,None,2.797473,-2.760736,-3.631893,0.135802,9.082088,-0.027788
...,...,...,...,...,...,...,...,...
360,2021-12-27,None,2.090360,-3.302426,-3.520976,-0.016915,9.082088,-0.151412
361,2021-12-28,None,2.301742,-3.303948,-3.541400,0.135802,9.082088,-0.070800
362,2021-12-29,None,2.167264,-3.305469,-3.559427,-0.115918,9.082088,0.065991
363,2021-12-30,None,2.357093,-3.306991,-3.575194,0.019338,9.082088,0.137852


In [29]:
forecast2 = m.predict(df=test)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.726% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - d
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.726% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - d


Predicting: 35it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [30]:
forecast2

,ds,y,yhat1,trend,season_yearly,season_weekly,season_daily,season_monthly
0,2021-01-01,2.070,2.891697,-2.754649,-3.603158,0.156855,9.082088,0.010560
1,2021-01-02,2.062,2.906502,-2.756171,-3.612601,0.100525,9.082088,0.092662
2,2021-01-03,1.918,2.607873,-2.757692,-3.620431,-0.279687,9.082088,0.183596
3,2021-01-04,3.238,2.837090,-2.759214,-3.626808,-0.016915,9.082088,0.157939
4,2021-01-05,2.864,2.797473,-2.760736,-3.631893,0.135802,9.082088,-0.027788
...,...,...,...,...,...,...,...,...
360,2021-12-27,3.830,2.090360,-3.302426,-3.520976,-0.016915,9.082088,-0.151412
361,2021-12-28,4.510,2.301742,-3.303948,-3.541400,0.135802,9.082088,-0.070800
362,2021-12-29,4.490,2.167264,-3.305469,-3.559427,-0.115918,9.082088,0.065991
363,2021-12-30,4.444,2.357093,-3.306991,-3.575194,0.019338,9.082088,0.137852
